In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("white")
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline
import functools

In [ ]:
import plotly.io as pio
import statsmodels.formula.api as sm
from scipy.optimize import minimize

# set default theme to seaborn for plotly.express
pio.templates.default = "simple_white"
# change default size
pio.templates[pio.templates.default].layout["height"] = 600
pio.templates[pio.templates.default].layout["width"] = 800
pio.templates["simple_white"].layout.autosize = False

## To do
- Do the example notebooks for
    - grid search [X]
    - derivative based line search 
    - db trust region
    - db free trust region
    - db free direct search

- Create quizzes in objectives materials for all the above.

# Visualize how classes of optimizers work

- All algorithms are very simplified compared to real algorithms
- All visualizations only use 1 dimensional function

## Some Tools

In [ ]:
def plot_function_plotly():
    x_grid = np.linspace(0, 20, 100)
    y_grid = [example_criterion(x) for x in x_grid]
    fig = px.line(x=x_grid, y=y_grid)
    # remove axis names
    fig.update_xaxes(title_text="")
    fig.update_yaxes(title_text="")
    return fig


def plot_function():
    x_grid = np.linspace(0, 20, 100)
    y_grid = [example_criterion(x) for x in x_grid]
    fig, ax = plt.subplots()
    sns.lineplot(x=x_grid, y=y_grid, ax=ax)
    sns.despine()
    return fig, ax


def plot_history(evaluated_points, argmin):
    """Plot the function and all evaluated points."""
    fig, ax = plot_function()
    sns.rugplot(evaluated_points, ax=ax)
    ax.plot([argmin], [example_criterion(argmin)], marker="*", color="firebrick")
    return fig, ax


def _check_argmin(argmin):
    if isinstance(argmin, np.ndarray):
        if len(argmin) == 1:
            argmin = argmin[0]
        else:
            raise ValueError("argmin should be a float or a np.ndarray of length 1")
    return argmin


def _fix_if_argmin_is_in_evaluated_points(evaluated_points, argmin):
    if argmin in evaluated_points:
        evaluated_points = evaluated_points[:-1]


def plot_history_plotly(evaluated_points, argmin):
    _fix_if_argmin_is_in_evaluated_points(evaluated_points, argmin)

    argmin = _check_argmin(argmin)

    x_grid = np.linspace(0, 20, 100)
    y_grid = [example_criterion(x) for x in x_grid]
    fig = px.line(x=x_grid, y=y_grid)
    # remove axis names
    fig.update_xaxes(title_text="")
    fig.update_yaxes(title_text="")

    fig.add_scatter(
        x=[argmin],
        y=[example_criterion(argmin)],
        mode="markers",
        marker={"size": 12, "color": "red"},
        name="Final evaluation",
        marker_symbol="star",
    )

    fig.add_scatter(
        x=evaluated_points,
        y=[example_criterion(x) for x in evaluated_points],
        mode="markers",
        marker={"size": 4, "color": "black"},
        name="Evaluations",
    )

    return fig


def minimize_with_history(fun, x0, method, jac=None, hess=None):
    """Dumbed down scipy minimize that returns full history.

    This is really only meant for illustration in this notebook. In particular,
    the following restrictions apply:

    - Only works for 1 dimensional problems
    - does not support all arguments

    """
    history = []

    def wrapped_fun(x, history=history):
        history.append(_unpack_x(x))
        return fun(x)

    res = minimize(wrapped_fun, x0, method=method, jac=jac, hess=hess)
    res.history = history
    return res


def taylor_expansion(x, x0, radius):
    """Evaluate taylor expansion around x0 at x."""
    x = _unpack_x(x)
    x0 = _unpack_x(x0)
    f = example_criterion(x0)
    f_prime = example_gradient(x0)
    f_double_prime = example_hessian(x0)

    if radius <= 0:
        raise ValueError("radius should be positive")

    diff = x - x0
    return f + f_prime * diff + f_double_prime * 0.5 * diff**2


def regression_surrogate(x, x0, radius):
    """Evaluate a regression based surrogate model at x.

    x0 and radius define the trust region in which evaluation points are sampled.

    """
    x = _unpack_x(x)
    x0 = _unpack_x(x0)
    deviations = [-radius, 0, radius]

    evaluations = [example_criterion(x0 + deviation) for deviation in deviations]
    df = pd.DataFrame()
    df["x"] = deviations
    df["y"] = evaluations
    params = sm.ols(formula="y ~ x + I(x**2)", data=df).fit().params
    vec = np.array([1, (x - x0), (x - x0) ** 2])
    return params @ vec


def plot_trust_region_algo(x0, radius, surrogate_func):
    fig, ax = plot_function()
    x0 = _unpack_x(x0)
    trust_x_grid = np.linspace(x0 - radius, x0 + radius, 50)
    partialed = functools.partial(surrogate_func, x0=x0, radius=radius)
    trust_y_grid = [partialed(x) for x in trust_x_grid]
    argmin_index = np.argmin(trust_y_grid)
    argmin = trust_x_grid[argmin_index]

    ax.plot([argmin], [partialed(np.array([argmin]))], marker="*", color="firebrick")
    ax.plot(
        [argmin],
        [example_criterion(np.array([argmin]))],
        marker="*",
        color="green",
    )

    sns.lineplot(x=trust_x_grid, y=trust_y_grid, ax=ax)

    new_x = (
        x0
        if example_criterion([argmin]) >= example_criterion(x0)
        else np.array([argmin])
    )

    if surrogate_func == taylor_expansion:
        x_values = [x0]
    else:
        x_values = [x0 - radius, x0, x0 + radius]

    sns.rugplot(x_values, ax=ax)
    return fig, ax, new_x


def plot_trust_region_algo_plotly(x0, radius, surrogate_func):
    fig = plot_function_plotly()
    x0 = _unpack_x(x0)
    trust_x_grid = np.linspace(x0 - radius, x0 + radius, 50)
    partialed = functools.partial(surrogate_func, x0=x0, radius=radius)
    trust_y_grid = [partialed(x) for x in trust_x_grid]
    argmin_index = np.argmin(trust_y_grid)
    argmin = trust_x_grid[argmin_index]

    fig.add_scatter(
        x=[argmin],
        y=[partialed(np.array([argmin]))],
        mode="markers",
        marker={"size": 12, "color": "red"},
        name="Approximate next evaluation",
        marker_symbol="star",
    )
    fig.add_scatter(
        x=[argmin],
        y=[example_criterion(np.array([argmin]))],
        mode="markers",
        marker={"size": 12, "color": "green"},
        name="True next evaluation",
        marker_symbol="star",
    )

    fig.add_scatter(
        x=trust_x_grid,
        y=trust_y_grid,
        mode="lines",
        name="Surrogate model",
        line={"color": "darkorange", "width": 2},
    )

    new_x = (
        x0
        if example_criterion([argmin]) >= example_criterion(x0)
        else np.array([argmin])
    )

    if surrogate_func == taylor_expansion:
        x_values = [x0]
    else:
        x_values = [x0 - radius, x0, x0 + radius]

    fig.add_scatter(
        x=x_values,
        y=[example_criterion(x) for x in x_values],
        mode="markers",
        marker={"size": 4, "color": "black"},
        name="Initial evaluation",
    )

    return fig, new_x


def plot_direct_search(x0, other):
    fig, ax = plot_function()
    x0 = _unpack_x(x0)
    other = _unpack_x(other)

    x_values = [x0, other]
    evaluations = [example_criterion(x) for x in x_values]

    argmin_index = np.argmin(evaluations)
    argmin = x_values[argmin_index]

    ax.plot([argmin], [example_criterion(argmin)], marker="*", color="firebrick")
    sns.rugplot(x_values, ax=ax)

    return fig, ax, argmin


def plot_direct_search_plotly(x0, other):
    fig = plot_function_plotly()
    x0 = _unpack_x(x0)
    other = _unpack_x(other)

    x_values = [x0, other]
    evaluations = [example_criterion(x) for x in x_values]

    argmin_index = np.argmin(evaluations)
    argmin = x_values[argmin_index]

    # remove argmin from x_values
    x_values = x_values[:argmin_index] + x_values[argmin_index + 1 :]

    fig.add_scatter(
        x=[argmin],
        y=[example_criterion(argmin)],
        mode="markers",
        marker={"size": 12, "color": "red"},
        name="Next evaluation",
        marker_symbol="star",
    )

    fig.add_scatter(
        x=x_values,
        y=[example_criterion(x) for x in x_values],
        mode="markers",
        marker={"size": 4, "color": "black"},
        name="Initial evaluation",
    )

    return fig, argmin


def plot_line_search(x0):
    fig, ax = plot_function()
    x0 = _unpack_x(x0)

    function_value = example_criterion(x0)
    gradient_value = example_gradient(x0)
    approx_hessian_value = np.clip(example_hessian(x0), 0.1, np.inf)
    base_step = -1 / approx_hessian_value * gradient_value

    gradient_grid = [x0 - 2, x0, x0 + 2]
    gradient_evals = [
        function_value - 2 * gradient_value,
        function_value,
        function_value + 2 * gradient_value,
    ]
    sns.lineplot(x=gradient_grid, y=gradient_evals, ax=ax)

    new_value = np.inf
    x_values = [x0]
    evaluations = [function_value]
    alpha = 1
    while new_value >= function_value:
        new_x = x0 + alpha * base_step
        new_value = example_criterion(new_x)
        x_values.append(new_x)
        evaluations.append(new_value)

    sns.rugplot(x_values, ax=ax)
    ax.plot([new_x], [new_value], marker="*", color="firebrick")
    return fig, ax, new_x


def plot_line_search_plotly(x0):
    fig = plot_function_plotly()
    x0 = _unpack_x(x0)

    function_value = example_criterion(x0)
    gradient_value = example_gradient(x0)
    approx_hessian_value = np.clip(example_hessian(x0), 0.1, np.inf)
    base_step = -1 / approx_hessian_value * gradient_value

    gradient_grid = [x0 - 2, x0, x0 + 2]
    gradient_evals = [
        function_value - 2 * gradient_value,
        function_value,
        function_value + 2 * gradient_value,
    ]
    # make it dark orange
    fig.add_scatter(
        x=gradient_grid,
        y=gradient_evals,
        mode="lines",
        name="Gradient",
        line={"color": "darkorange", "width": 2},
    )

    new_value = np.inf
    x_values = [x0]
    evaluations = [function_value]
    alpha = 1
    while new_value >= function_value:
        new_x = x0 + alpha * base_step
        new_value = example_criterion(new_x)
        x_values.append(new_x)
        evaluations.append(new_value)

    # mark the point of the initial evaluation with a circle
    fig.add_scatter(
        x=[x0],
        y=[function_value],
        mode="markers",
        marker={"size": 8, "color": "green"},
        name="Initial point",
    )

    # mark the point of the next step with a star
    fig.add_scatter(
        x=[new_x],
        y=[new_value],
        mode="markers",
        marker={"size": 12, "color": "blue"},
        name="Next initial point",
        marker_symbol="star",
    )

    # mark the true minimum with a star
    fig.add_scatter(
        x=[argmin],
        y=[example_criterion(np.array([argmin]))],
        mode="markers",
        marker={"size": 12, "color": "red"},
        name="Global minimum",
        marker_symbol="star",
    )

    return fig, new_x

## Set up a function

- The function has a local minimum at 9.594 and a global minimum at 17.37
- It is twice continuously differentiable
- It is not convex
- In some areas it can be reasonably well approximated by a quadratic function, it others it cannot

In [ ]:
WEIGHTS = [
    9.003014962148157,
    -3.383000146393776,
    -0.6037887934635748,
    1.6984454347036886,
    -0.9447426232680957,
    0.2669069434366247,
    -0.04446368897497234,
    0.00460781796708519,
    -0.0003000790127508276,
    1.1934114174145725e-05,
    -2.6471293419570505e-07,
    2.5090819960943964e-09,
]


def example_criterion(x):
    x = _unpack_x(x)
    exponents = np.arange(len(WEIGHTS))
    return WEIGHTS @ x**exponents


def example_gradient(x):
    x = _unpack_x(x)
    exponents = np.arange(len(WEIGHTS))
    return (WEIGHTS * exponents) @ x ** (exponents - 1).clip(0)


def example_hessian(x):
    x = _unpack_x(x)
    exponents = np.arange(len(WEIGHTS))
    return (WEIGHTS * exponents * (exponents - 1)) @ x ** (exponents - 2).clip(0)


def _unpack_x(x):
    if hasattr(x, "__len__"):
        assert len(x) == 1

    if isinstance(x, pd.DataFrame):
        res = x["value"].to_numpy()[0]
    elif isinstance(x, pd.Series):
        res = x.to_numpy()[0]
    elif isinstance(x, np.ndarray | list | tuple):
        res = x[0]
    else:
        res = float(x)
    return res


start_x = np.array([2])
fig = plot_function()
fig

## Grid Search

- Needs bounds on the parameter (0 to 20 in our case)
- Desired precision determines number of grid points
- Very feasible in one dimension

In [ ]:
n_points = 100
grid = np.linspace(0, 20, n_points)
evaluations = [example_criterion(x) for x in grid]
argmin_index = np.argmin(evaluations)
argmin = grid[argmin_index]

fig = plot_history_plotly(grid, argmin)
fig

## Derivative based line search algorithm

### Basic Idea

- Use first derivative to get search direction
- Use approximated second derivative to guess step length
- Use a line search algorithm to see how far to go in the search direction
    - Line search stays a 1d problem even with many parameters
    - Only solved approximately
    - Quite complicated if you really want to understand it
    - Most of the time accepts the first guess
- Accept the new parameter and go back to start

### Ilustration

This is just an illustration of the principle. The Hessian approximation is just the real Hessian clipped at 0.1 so it works for non-convex functions. This is neither smart, nor computationally feasible for large problems. The line search accepts the initial guess if it yields an improvement and shortens the step length otherwise. While this is guaranteed to yield an improvement at some point, it is not a smart line search algorithm. Real algorithms approximate the hessian in a better way and use a smarter line search algorithm. You should never implement an optimization algorithm yourself. Leave it to experts unless you are a very good programmer, have read a few books on optimization algorithms and a very special problem that cannot be solved with existing algorithms. 

### Initial evaluation

In [ ]:
fig, new_x = plot_line_search_plotly(start_x)
fig

- large gradient, low curvature
- make a big step

### Iteration 1

In [ ]:
fig, x = plot_line_search_plotly(new_x)
fig

- large gradient, large curvature
- make a smaller step

### Iteration 2

In [ ]:
fig, x = plot_line_search_plotly(x)
fig

- very small gradient, low curvature
- make a very small step

### Iteration 3

In [ ]:
# step is very small, due to low gradient and low curvature
fig, x = plot_line_search_plotly(x)
fig

- very small gradient, low curvature
- make a very small step

### Iteration 4

In [ ]:
fig, x = plot_line_search_plotly(x)
fig

- medium-sized gradient, low curvature
- make a larger step again

### Iteration 5

In [ ]:
fig, x = plot_line_search_plotly(x)
fig

- medium-sized gradient, larger curvature 
- make a small step

### Iteration 6

In [ ]:
fig, x = plot_line_search_plotly(x)
fig

- reverse direction due to gradient

### Iteration 7

In [ ]:
fig, x = plot_line_search_plotly(x)
fig

convergence based on gradient ≈ zero criterion

### Notes

- A big advantage over algorithms you will see later is that this has no tuning parameters.
- Using hessian for step length is much better than standard gradient descent. 
- In very high dimensional problems, standard gradient descent can nevertheless be computationally better.

### A real algorithm: L-BFGS-B

In [ ]:
res = minimize_with_history(
    example_criterion,
    start_x,
    method="L-BFGS-B",
    jac=example_gradient,
)
len(res.history)

In [ ]:
fig = plot_history_plotly(res.history, res.x)
fig

## Derivative based trust region algorithm


### Basic Idea

1. Fix a trust region radius
1. Construct a Taylor expansion of the function based on function value, gradient, and (approximation to) Hessian 
  
   The Taylor expansion
   - approximates the function well within trust region if radius is not too large
   - is a quadratic function that it easy to optimize 

1. Minimize the Taylor expansion within the trust region
1. Evaluate function again at the argmin of the Taylor expansion
1. Compare expected and actual improvement
1. Accept the new parameters if actual improvement is good enough
1. Potentially modify the trust region radius 
  
   **This is a very important and very complicated step**

1. Go back to 2.


## Illustration

This is just an illustration of the principle. The trust region radius is updated manually to simulate a real algorithm.

Note that a real algorithm is quite complex to implement. You should never do that yourself. Leave it to experts unless
- you are a very good programmer,
- you have read a few books on optimization algorithms,
-  and you have a very special problem that cannot be solved with existing algorithms. 

### Initial evaluation

In [ ]:
fig, x_0 = plot_trust_region_algo_plotly(start_x, 2, taylor_expansion)
fig

- orange line has approximation
- expected improvement: difference between f(x0) and minimum of the orange line
- actual improvement: difference beween f(x0) and f(x)
- since actual improvement / expected improvement large (>1 even)
  - accept the new point → midpoint of new trust region
  - increase trust region radius
  
### Iteration 1

In [ ]:
fig, x_1 = plot_trust_region_algo_plotly(x_0, 3, taylor_expansion)
fig

- actual improvement / expected improvement < 1, but still large
- accept new point, increase trust region

### Iteration 2

In [ ]:
fig, x_2 = plot_trust_region_algo_plotly(x_1, 4, taylor_expansion)
fig

- actual improvement / expected improvement is negative
- Do not new accept point
- Decrease trust region radius

### Iteration 3

In [ ]:
fig, x_3 = plot_trust_region_algo_plotly(x_2, 2, taylor_expansion)
fig

- actual improvement / expected improvement around 1
- accept new point, increase trust region

### Iteration 4

In [ ]:
fig, x_4 = plot_trust_region_algo_plotly(x_3, 3, taylor_expansion)
fig

- actual improvement / expected improvement around 1
- accept new point, increase trust region

### Iteration 5

In [ ]:
fig, x_5 = plot_trust_region_algo_plotly(
    x_4,
    radius=3,
    surrogate_func=taylor_expansion,
)
fig

- Convergence somewhere here


### Notes

- Most of the time, the approximation was not very good but sent us in the right direction
- After a successful iteration, the trust region radius is increased
- At some point it becomes too large and needs to be decreased
- From now on the algorithm would converge soon because of a zero gradient
- Even when it converges, the trust region radius does not shrink to zero

### A real algorithm: Trust-NCG

In [ ]:
res = minimize_with_history(
    example_criterion,
    start_x,
    method="trust-ncg",
    jac=example_gradient,
    hess=example_hessian,
)
fig = plot_history_plotly(res.history, res.x)
fig

In [ ]:
res = minimize_with_history(
    example_criterion,
    start_x,
    method="trust-ncg",
    jac=example_gradient,
    hess=example_hessian,
)
plot_history(res.history, res.x)
len(res.history)

## Derivative free trust region algorithm

### Basic Idea

- Similar to derivative based trust region algorithm.
- Instead of Taylor expansion, use a surrogate model based on interpolation or regression.
    - Interpolation: Function is evaluated at exactly as many points as you need to fit the model.
    - Regression: Function is evaluated at more points than you strictly need. Better for noisy functions.
    - In general: Evaluation points are spread further out than for numerical derivatives.
- How the evaluation points are determined is complicated. It is also crucial for the efficiency of the algorithm.

## Illustration

This is just an illustration of the principle. The trust region radius is updated manually to simulate a real algorithm.

Note that a real algorithm is quite complex to implement. You should never do that yourself. Leave it to experts unless
- you are a very good programmer,
- you have read a few books on optimization algorithms,
-  and you have a very special problem that cannot be solved with existing algorithms. 

### Initial evaluation

In [ ]:
fig, ax, x = plot_trust_region_algo(
    start_x,
    radius=2,
    surrogate_func=regression_surrogate,
)

- orange line has approximation
- expected improvement: difference between f(x0) and minimum of the orange line
- actual improvement: difference beween f(x0) and f(x)
- since actual improvement / expected improvement large (around 1)
  - accept the new point → midpoint of new trust region
  - increase trust region radius
  
### Iteration 1

In [ ]:
fig, ax, x = plot_trust_region_algo(x, radius=3, surrogate_func=regression_surrogate)

- actual improvement / expected improvement large (around 1) again
- accept new point, increase trust region

### Iteration 2

In [ ]:
fig, ax, x = plot_trust_region_algo(x, radius=4, surrogate_func=regression_surrogate)

- actual improvement / expected improvement large (around 1)
- but step length is small
- accept new point, decrease trust region

### Iteration 3

In [ ]:
fig, ax, x = plot_trust_region_algo(x, radius=2, surrogate_func=regression_surrogate)

- actual improvement / expected improvement reasonable
- step length is small
- accept new point, decrease trust region

### Iteration 4

In [ ]:
fig, ax, x = plot_trust_region_algo(x, radius=1, surrogate_func=regression_surrogate)

- actual improvement / expected improvement large (around 1)
- step length is large
- accept new point, increase trust region

### Iteration 5+

- will continue around here until optimum is reached
- you get the idea


### Notes

- The fit is generally better than the gradient based trust region algorithm
- By construction especially at corners of trust region
- Choose between the two based on computation speed
    - If you have fast closed form derivatives, use the derivative based algorithm
    - If you only have numerical derivatives, use this instead
- Points at which the function has been evaluated before can be re-used to save function evaluations
- Since no gradient is available, algorithm will continue until trust region radius shrinks to zero
- It's intuitively very clear how this can work for noisy functions if enough evaluations are used for each surrogate model


### A real algorithm: Cobyla

In [ ]:
res = minimize_with_history(example_criterion, start_x, method="Cobyla")
plot_history(res.history, res.x)
len(res.history)

## Derivative free direct search algorithm

### Basic Idea

- Explore parameter space around current point systematically and accept the best value
- Also called pattern search because the points at which the function is evaluated form a pattern
- Easiest example for one dimensional problems:
    - Evaluate function at current point and one other point
    - Switch direction of other point if you got a decrease
    - Make steps larger after success
    - Make steps smaller after failure


## Illustration

This is just an illustration of the principle. The trust region radius is updated manually to simulate a real algorithm.

Note that a real algorithm is quite complex to implement. You should never do that yourself. Leave it to experts unless
- you are a very good programmer,
- you have read a few books on optimization algorithms,
-  and you have a very special problem that cannot be solved with existing algorithms. 

### Initial evaluation

In [ ]:
fig, x_0 = plot_direct_search_plotly(start_x, start_x - 2)
fig

In [ ]:
x = start_x
fig, ax, x = plot_direct_search(x, x - 2)

- candidate value worse than original value
- do not accept candidate value, switch direction

### Iteration 1

In [ ]:
fig, ax, x = plot_direct_search(x, x + 2)

In [ ]:
fig, x_1 = plot_direct_search_plotly(x_0, x_0 + 2)
fig

- candidate value better than original value
- accept candidate value, increase step length

### Iteration 2

In [ ]:
fig, x_2 = plot_direct_search_plotly(x_1, x_1 + 3)
fig

In [ ]:
fig, ax, x = plot_direct_search(x, x + 3)

- candidate value better than original value
- accept candidate value, increase step length

### Iteration 3

In [ ]:
fig, ax, x = plot_direct_search(x, x + 4)

- candidate value worse than original value
- do not accept new point, make step smaller

### Iteration 4

In [ ]:
fig, ax, x = plot_direct_search(x, x + 2)

- from here on it becomes complex because we already know from iteration 3 that we will do worse further right
- will eventually converge around here

### Notes

- Adjusting the step size and switching to promising directions is complicated in real algorithms
- Direct search algorithms only use the information which function value is smallest, not by how much
- Makes them slow but robust to small amounts of noise
- It does not help for large amounts of noise
- Most famous example is the Nelder-Mead algorithm which is widely used, but seldomly the best choice


### A real example: Nelder Mead:

In [ ]:
res = minimize_with_history(example_criterion, start_x, method="Nelder-Mead")
plot_history(res.history, res.x)
len(res.history)